In [1]:
# Bone Fracture Detection using Deep Learning (CNN)
# This project uses deep learning and computer vision to classify bone fractures from X-ray images.

import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import time

# Display TensorFlow version and GPU availability
print("TensorFlow Version:", tf.__version__)
print("GPU Availability:", tf.config.list_physical_devices('GPU'))
from tensorflow.python.platform import build_info as tf_build_info
print("CUDA Support:", tf_build_info.build_info.get("cuda_version", "Not compiled with CUDA"))
print("cuDNN Support:", tf_build_info.build_info.get("cudnn_version", "Not compiled with cuDNN"))

# Set up paths for dataset
project_dir = os.getcwd().replace("code", "dataset")
print(f"Dataset Directory: {project_dir}")
train_dir = os.path.join(project_dir, "train")
test_dir = os.path.join(project_dir, "test")
val_dir = os.path.join(project_dir, "val")

# Verify dataset contents
print("Training Directory Contents:", os.listdir(train_dir))
print("Test Directory Contents:", os.listdir(test_dir))
print("Validation Directory Contents:", os.listdir(val_dir))

# Function to rename files, replacing spaces with underscores
def rename_files_in_directory(directory):
    """
    Rename files by replacing spaces with underscores in the directory and subdirectories.
    """
    for root, _, files in os.walk(directory):
        for file in files:
            if ' ' in file:
                old_path = os.path.join(root, file)
                new_file = file.replace(' ', '_')
                new_path = os.path.join(root, new_file)
                os.rename(old_path, new_path)
                print(f'Renamed: "{old_path}" → "{new_path}"')

# Rename files in train, test, and validation directories
rename_files_in_directory(os.path.join(train_dir, "fractured"))
rename_files_in_directory(os.path.join(train_dir, "not_fractured"))
rename_files_in_directory(os.path.join(test_dir, "fractured"))
rename_files_in_directory(os.path.join(test_dir, "not_fractured"))
rename_files_in_directory(os.path.join(val_dir, "fractured"))
rename_files_in_directory(os.path.join(val_dir, "not_fractured"))

# Function to remove invalid or corrupted images
def clean_invalid_images(directory):
    invalid_count = 0
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                img = tf.io.read_file(file_path)
                tf.image.decode_image(img)  # Try decoding the image
            except tf.errors.InvalidArgumentError:
                print(f"Invalid Image Removed: {file_path}")
                os.remove(file_path)
                invalid_count += 1
    print(f"Removed {invalid_count} invalid images from {directory}")

# Clean invalid images from all directories
clean_invalid_images(os.path.join(train_dir, "fractured"))
clean_invalid_images(os.path.join(train_dir, "not_fractured"))
clean_invalid_images(os.path.join(test_dir, "fractured"))
clean_invalid_images(os.path.join(test_dir, "not_fractured"))
clean_invalid_images(os.path.join(val_dir, "fractured"))
clean_invalid_images(os.path.join(val_dir, "not_fractured"))

# Set batch size and image size
batch_size = 32
image_dimensions = (160, 160)

# Create datasets from directories
train_data = tf.keras.utils.image_dataset_from_directory(train_dir, shuffle=True, batch_size=batch_size, image_size=image_dimensions)
test_data = tf.keras.utils.image_dataset_from_directory(test_dir, shuffle=True, batch_size=batch_size, image_size=image_dimensions)
val_data = tf.keras.utils.image_dataset_from_directory(val_dir, shuffle=True, batch_size=batch_size, image_size=image_dimensions)

# Class names (fractured vs not fractured)
class_names = train_data.class_names
print("Class Names:", class_names)

# Visualizing some sample images from the training set
plt.figure(figsize=(10, 10))
for images, labels in train_data.take(2):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.suptitle("Training Dataset Samples", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

# Prefetch the datasets for optimization
train_data = train_data.prefetch(buffer_size=tf.data.AUTOTUNE)
val_data = val_data.prefetch(buffer_size=tf.data.AUTOTUNE)
test_data = test_data.prefetch(buffer_size=tf.data.AUTOTUNE)

# Image Augmentation
aug_flip = tf.keras.layers.RandomFlip('horizontal')
aug_rotate = tf.keras.layers.RandomRotation(0.02)
augmentation_pipeline = tf.keras.Sequential([aug_flip, aug_rotate])

# Display some augmented images
plt.figure(figsize=(12, 12))
for images, _ in train_data.take(1):
    sample_image = images[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_img = augmentation_pipeline(tf.expand_dims(sample_image, 0))
        plt.imshow(augmented_img[0].numpy().astype("uint8"))
        plt.axis('off')
        plt.title(f"Augmented Image {i+1}", fontsize=12)
plt.suptitle("Sample Augmented Images", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

# Transfer Learning with MobileNetV2
input_shape = image_dimensions + (3,)  # RGB images
base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

# Building the final model
inputs = tf.keras.Input(shape=input_shape)
x = augmentation_pipeline(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)  # Optional dropout layer for regularization
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)
model.summary()

# Compile the model
learning_rate_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.1, decay_steps=batch_size, decay_rate=0.8)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule), loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping and model checkpoint
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint_callback = ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)

# Training the model
start_time = time.time()
history = model.fit(train_data, epochs=20, validation_data=val_data, callbacks=[early_stopping_callback, model_checkpoint_callback], verbose=2)

# Save the training history
with open('training_history.json', 'w') as file:
    json.dump(history.history, file)

# Evaluate the model
model = tf.keras.models.load_model('best_model.h5')
test_loss, test_acc = model.evaluate(test_data)

# Plot training and validation accuracy & loss
with open('training_history.json', 'r') as file:
    history_data = json.load(file)

epochs_range = np.arange(len(history_data['accuracy']))
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Plot accuracy
ax[0].plot(epochs_range, history_data['accuracy'], label='Training Accuracy')
ax[0].plot(epochs_range, history_data['val_accuracy'], label='Validation Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epochs')
ax[0].legend()

# Plot loss
ax[1].plot(epochs_range, history_data['loss'], label='Training Loss')
ax[1].plot(epochs_range, history_data['val_loss'], label='Validation Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epochs')
ax[1].legend()

plt.tight_layout()
plt.show()

# End of script


ModuleNotFoundError: No module named 'numpy'

In [ ]:
import sys
print(sys.executable)
print(sys.version)
